In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go

from scipy.spatial import Delaunay

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

path = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=11.5eV' # use your path
all_files = glob.glob(path + "/*.dat")

li = []
colnames=["phi","cos(theta)","value"] 

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None) 
    # or delim_whitespace=True, it is faste
    # r"\s+" is a regex (regular expression)
    df["filename"]=filename.split("\\")[-1].split(".")[0] # adding a column with the file name
    df["filename"]=df["filename"].str.replace(" ","") # corrects for spaces
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

frame = pd.concat(li, axis=0)
frame_srt=frame.groupby("filename")
frame.loc[frame["filename"] == "1_1"] # first way of selecting the right file
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
counts = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # counts

#Forcing the values at the edges of +-phi to be the same avg
test11=frame.loc[frame["filename"] == "1_1"]
a1=test11.loc[test11["phi"] == np.min(phi)]["value"].to_numpy()
b1=test11.loc[test11["phi"] == np.max(phi)]["value"].to_numpy()
avgphi=np.divide(a1+b1,2)
avgphi=np.pad(avgphi, (0,len(phi)-len(avgphi)), 'constant', constant_values=0)
# avgphipd = pd.Series(avgphi)
test11["value"].mask(test11["phi"] == np.min(phi), avgphi, inplace=True)
test11["value"].mask(test11["phi"] == np.max(phi), avgphi, inplace=True)

#Forcing the values at the edges of +-ctheta to be the same avg
a2=test11.loc[test11["cos(theta)"] == np.min(ctheta)]["value"].to_numpy()
b2=test11.loc[test11["cos(theta)"] == np.max(ctheta)]["value"].to_numpy()
avgcos=np.divide(a2+b2,2)
avgcos=np.pad(avgcos, (0,len(ctheta)-len(avgcos)), 'constant', constant_values=0)
# avgcospd = pd.Series(avgcos)
# test11["value"].mask(test11["cos(theta)"] == np.min(ctheta), avgcos, inplace=True)
# test11["value"].mask(test11["cos(theta)"] == np.max(ctheta), avgcos, inplace=True)

counts_fixed = test11.iloc[:,2].to_numpy()
phi_fixed = np.where(phi==np.min(phi), -180., phi) #the last is an else
phi_fixed = np.where(phi==np.max(phi), 180., phi_fixed)
ctheta_fixed = np.where(ctheta==np.min(ctheta), -1., ctheta)
ctheta_fixed = np.where(ctheta==np.max(ctheta), 1., ctheta_fixed)
# print(np.unique(phi_fixed))

# theta_rad = np.arccos(ctheta)
theta_rad = np.arccos(ctheta_fixed)
# phi_rad = phi * np.pi/180.
phi_rad = phi_fixed * np.pi/180.
# countsn = counts
countsn = counts_fixed

# convertion to spherical coordinates 1D vectors of shape (2000,1)
x = countsn * np.sin(theta_rad) * np.cos(phi_rad)
X = x.reshape(100,200)
y = countsn * np.sin(theta_rad) * np.sin(phi_rad)
Y = y.reshape(100,200)
z = countsn * ctheta
Z = z.reshape(100,200)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

In [ ]:
#TRIANGULATION
def triangulation_edges(points, faces, linewidth=1.5):
    points = np.asarray(points)
    faces = np.asarray(faces)
    d = points.shape[-1]
    
    if d not in [2, 3] or faces.shape[-1] != 3:
        raise ValueError("your data are not associated to a 2d or 3d  triangulation\n\
                         points should be an array of ndim=2 or 3 and faces of ndim=3")
    
    tri_vertices = points[faces]
    Xe = []
    Ye = []
    if d == 3:
        Ze = []
    for T in tri_vertices:
        Xe += [T[k%3][0] for k in range(4)] + [None]
        Ye += [T[k%3][1] for k in range(4)] + [None]
        if d == 3: 
            Ze += [T[k%3][2] for k in range(4)] + [None] 
    if d == 2:
        return  go.Scatter(x=Xe,
                           y=Ye,
                           mode='lines',
                           name='edges',
                           line_color ='rgb(50,50,50)', 
                           line_width=linewidth
                            )
    else:
        return go.Scatter3d(x=Xe,
                           y=Ye,
                           z=Ze,
                           mode='lines',
                           name='edges',
                           line_color ='rgb(50,50,50)', 
                           line_width=linewidth)

In [ ]:
points2d_trace = go.Scatter(x=x, y=y, mode='markers', marker_color='red', marker_size=6)
point_trace= go.Scatter(x=x, y=y, 
                         mode='markers',
                         name='points',
                         marker_color='red',
                         marker_size=6)
pts2d = np.array([x,y]).T
tri = Delaunay(pts2d)
delaunay_tri = triangulation_edges(pts2d, tri.simplices, linewidth=1)
fig2 = go.Figure(data=[delaunay_tri, point_trace])
fig2.update_layout(width=500, height=500);
# fig2.show()

In [ ]:
pts3d = np.array([x,y,z]).T
pts2d = np.array([x,y]).T
pts3d.shape[-1]

In [ ]:
points3d_trace = go.Scatter3d(x=x, y=y, z=z, mode='markers', marker_color='red', marker_size=6)
point_trace3d= go.Scatter3d(x=x, y=y, z=z,
                         mode='markers',
                         name='points',
                         marker_color='red',
                         marker_size=6)

In [ ]:
tritri = Delaunay(pts3d)
tri = Delaunay(pts2d)

tri.simplices

In [ ]:
delaunay_tritri = triangulation_edges(pts3d, tri.simplices, linewidth=1)
fig4 = go.Figure(data=[delaunay_tritri, point_trace3d])
fig4.update_layout(width=500, height=500);
fig4.show()

In [ ]:
i, j, k = tri.simplices.T
my_mesh3d = go.Mesh3d(
    x = x,
    y = y,
    z = z,
    i=i, j=j, k=k,
    colorscale='deep_r',
    colorbar_thickness=25,
    intensity=z, 
    flatshading=True
)

my_layout = dict(width=800, height=600,
                 scene_camera_eye=dict(x=1.5, y=1.5, z=1), 
                 scene_aspectratio=dict(x=1.5, y=1.5, z=1))

points3d = np.array([x,y,z]).T

delaun_tri3d  = triangulation_edges(points3d, tri.simplices)

fig3d = go.Figure(data=[my_mesh3d, delaun_tri3d])

# fig3d.show()

In [ ]:
from pyntcloud import PyntCloud, structures

pts = np.array([x, y, z]).T

delaun = structures.Delaunay3D(pts)

delaun.compute()
mesh = delaun.get_mesh()
tri= mesh[['v1', 'v2', 'v3']].values
I, J, K = tri.T


fig1 = go.Figure(go.Mesh3d(x=x, y=y, z=z, 
                           i=I, j=J, k=K, 
                           intensity=z, 
                           colorscale='Viridis', 
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig1.update_layout(width=800, height=800)
# fig1.show()